In [0]:
# Code to read file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticating email
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
import logging
logging.getLogger('googleapicliet.discovery_cache').setLevel(logging.ERROR)
#2.1 Get the file
downloaded = drive.CreateFile({'id':'1O7VHt4nDRi6UC5TbEZpv2ABsA-o4mEOV'}) # replace the id with id of file you want to access
downloaded.GetContentFile('Credit_Card_Applications.csv')


In [0]:
# Self Organizing Map

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing the dataset
dataset = pd.read_csv('Credit_Card_Applications.csv')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

In [0]:
dataset.head()

In [0]:
# Feature Scaling
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
X = sc.fit_transform(X)

In [0]:
!pip install MiniSom

In [0]:
#TRAINING THE SOM:
#this code made by a developer, an implementation of SOM:
from minisom import MiniSom

som = MiniSom(x = 10, y = 10, input_len = 15, sigma = 1.0, learning_rate = 0.5)
#x and y are the dimensions: we can choose whatever dimensions we want
#10 by 10 grid
#input lane: the number of features we have in the X. 14 attributes + customer ID
#sigma: is the radius

som.random_weights_init(X)
#we initialize the weights

som.train_random(data = X, num_iteration = 100)
#we train the algorithm

In [0]:
# Visualizing the results
from pylab import bone, pcolor, colorbar, plot, show
bone()
pcolor(som.distance_map().T)
colorbar()
markers = ['o', 's']
colors = ['r', 'g']
for i, x in enumerate(X):
    w = som.winner(x)
    plot(w[0] + 0.5,
         w[1] + 0.5,
         markers[y[i]],
         markeredgecolor = colors[y[i]],
         markerfacecolor = 'None',
         markersize = 10,
         markeredgewidth = 2)
show()

* From the result we can observe that the dark nodes are the wining nodes and the darker shaded nodes surrounding them are the ones which are closely representing the wining node features. while the white blocks represent 
*   Red circles depict the customers who did not get approval and green represents customers who got approval from the bank
*   Circles represent customers with No approval and Square represents customers with Aproval. 



In [0]:
#FINDING THE FRAUDS:
mappings = som.win_map(X)

frauds = np.concatenate((mappings[(5,2)], mappings[(6,4)], mappings[(5,4)]), axis = 0)
frauds = sc.inverse_transform(frauds)
frauds_df =pd.DataFrame(frauds)
frauds_df
#column number 0 is the customer ID.
#the bank can investigate more about those customers to catch potential fraud

In [0]:
# GET white and light coordinates:
mapping2 = som.distance_map().T
all_coordinates = {}
i_counter = 0
for i in mapping2:
    counter_x = 0
    for x in i:
        value = x
        coordinate = (counter_x,i_counter)
        all_coordinates[coordinate] = value
        counter_x += 1
    i_counter += 1
white = []
for x in all_coordinates.keys():
    if all_coordinates[x] >= 0.80:
        white.append(x)
white